## Demonstration of the User-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing user interactions, such as scroll length and read time, to identify users with similar behavior. 
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.



In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from utils.data_preprocessing import DataProcesser
from models.collaborative.user_based_CF import UserBasedCollaborativeRecommender

pl.Config.set_tbl_cols(-1)

polars.config.Config

## Data Import and EDA

In [2]:
data_reader = ParquetDataReader()
articles_df = data_reader.read_data('../../data/articles.parquet')
train_behaviors_df = data_reader.read_data('../../data/train/behaviors.parquet')
train_history_df = data_reader.read_data('../../data/train/history.parquet')
document_vectors_df = data_reader.read_data('../../data/document_vector.parquet')

We check the size of the data. From the sizes we learn that:
<ol>
  <li>We have 20738 unique articles</li>
  <li>We have 15143 unique users</li>
  <li>We have 232887 interactions in the testset</li>
</ol> 

In [3]:
print("articles_df has the size:         ", articles_df.shape)
print("train_behaviors_df has the size:  ", train_behaviors_df.shape)
print("train_history_df has the size:    ", train_history_df.shape)
print("document_vectors_df has the size: ", document_vectors_df.shape)

articles_df has the size:          (20738, 21)
train_behaviors_df has the size:   (232887, 17)
train_history_df has the size:     (15143, 5)
document_vectors_df has the size:  (125541, 2)


### Validation Set

In [4]:
test_behaviours_df = data_reader.read_data('../../data/validation/behaviors.parquet')
test_behaviours_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
96791,null,2023-05-28 04:21:24,9.0,null,2,"[9783865, 9784591, … 9784710]",[9784696],22548,false,null,null,null,false,142,72.0,100.0
96798,null,2023-05-28 04:31:48,46.0,null,2,"[9782884, 9783865, … 9784648]",[9784281],22548,false,null,null,null,false,143,16.0,28.0
96801,null,2023-05-28 04:30:17,14.0,null,2,"[9784648, 7184889, … 9781983]",[9784444],22548,false,null,null,null,false,143,12.0,24.0
96808,null,2023-05-28 04:27:19,22.0,null,2,"[9784607, 9695098, … 9781983]",[9781983],22548,false,null,null,null,false,142,125.0,80.0
96810,null,2023-05-28 04:29:47,23.0,null,2,"[9781983, 7184889, … 9781520]",[9784642],22548,false,null,null,null,false,142,null,null


In [5]:
# Combine train and test behaviors
combined_df = pl.concat([train_behaviors_df, test_behaviours_df])

# Generate a random mask for splitting
n = combined_df.height  # Total number of rows
test_mask = np.random.rand(n) < 0.30  # 30% test, 70% train

# Apply the mask
test_behaviors_df = combined_df.filter(test_mask)
train_behaviors_df = combined_df.filter(~test_mask)

# Verify the split
print(f"Train size: {train_behaviors_df.shape[0]}, Test size: {test_behaviors_df.shape[0]}")

Train size: 334250, Test size: 143284


### Table Contents

The information on news articles. As we are going to perform user-user CF, this table is not neccesary

In [6]:
articles_df.head()

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""
3012771,"""Morten Bruun fyret i Sønderjys…","""FODBOLD: Morten Bruun fyret me…",2023-06-29 06:20:39,false,"""Kemien mellem spillerne i Supe…",2006-05-01 14:28:40,[3177953],"""article_default""","""https://ekstrabladet.dk/sport/…",[],[],"[""Erhverv"", ""Kendt"", … ""Ansættelsesforhold""]",142,"[196, 199]","""sport""",null,null,null,0.8241,"""Negative"""
3023463,"""Luderne flytter på landet""","""I landets tyndest befolkede om…",2023-06-29 06:20:43,false,"""Det frække erhverv rykker på l…",2007-03-24 08:27:59,[3184029],"""article_default""","""https://ekstrabladet.dk/nyhede…",[],[],"[""Livsstil"", ""Erotik""]",118,[133],"""nyheder""",null,null,null,0.7053,"""Neutral"""
3032577,"""Cybersex: Hvornår er man utro?""","""En flirtende sms til den flott…",2023-06-29 06:20:46,false,"""De fleste af os mener, at et t…",2007-01-18 10:30:37,[3030463],"""article_default""","""https://ekstrabladet.dk/sex_og…",[],[],"[""Livsstil"", ""Partnerskab""]",565,[],"""sex_og_samliv""",null,null,null,0.9307,"""Neutral"""


Each file consists of seven days of impression logs. The train_behaviors_df table contains all information about interactions between users and items, and can be used as a basis for user-user CF. <strong>Therefore we only need this table</strong>.

In [7]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
150528,null,2023-05-24 07:33:25,25.0,null,2,"[9778718, 9778728, … 9778682]",[9778623],143471,false,null,null,null,false,1240,287.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
155586,9777492,2023-05-24 07:54:15,134.0,100.0,1,"[9486486, 9777397, … 9775983]",[9778627],161621,false,null,null,null,false,3625,50.0,100.0
155588,9778769,2023-05-24 07:51:14,24.0,100.0,1,"[9775716, 9778745, … 9778718]",[9778718],161621,false,null,null,null,false,3625,59.0,100.0


Each file consists of users' click histories collected over 21 days period. This table does contain the same values as the train_behaviours_df, but as that table is easier to work with we will use train_behaviours_df over this one

In [8]:
train_history_df.head()

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"
20396,"[2023-04-27 12:30:44, 2023-04-27 12:31:34, … 2023-05-17 10:59:44]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]"
34912,"[2023-04-29 07:12:49, 2023-04-29 13:01:18, … 2023-05-18 05:06:40]","[100.0, 35.0, … 27.0]","[9741802, 9741804, … 9770882]","[153.0, 7.0, … 5.0]"
37953,"[2023-04-27 19:17:10, 2023-04-27 19:17:27, … 2023-05-17 21:29:22]","[14.0, 28.0, … 18.0]","[9739205, 9739202, … 9769306]","[4.0, 16.0, … 6.0]"


List of vectors for each article. This is used to describe the items. It could be used for item-item CF, but is not relevant to user-user CF.  <strong>This table will therefore not be used</strong>

In [9]:
document_vectors_df.head()

article_id,document_vector
i32,list[f32]
3000022,"[0.065424, -0.047425, … 0.035706]"
3000063,"[0.028815, -0.000166, … 0.027167]"
3000613,"[0.037971, 0.033923, … 0.063961]"
3000700,"[0.046524, 0.002913, … 0.023423]"
3000840,"[0.014737, 0.024068, … 0.045991]"


From the analasys we see that we only need train_behaviour_df to perform user-user CF

## Preprocessing

### Remove Non-Needed Values

We see that we have several items that are not required for performing user-user CF

In [10]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
150528,null,2023-05-24 07:33:25,25.0,null,2,"[9778718, 9778728, … 9778682]",[9778623],143471,false,null,null,null,false,1240,287.0,100.0
153070,9777492,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, … 9778628]",[9778628],151570,false,null,null,null,false,1976,4.0,18.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
155586,9777492,2023-05-24 07:54:15,134.0,100.0,1,"[9486486, 9777397, … 9775983]",[9778627],161621,false,null,null,null,false,3625,50.0,100.0
155588,9778769,2023-05-24 07:51:14,24.0,100.0,1,"[9775716, 9778745, … 9778718]",[9778718],161621,false,null,null,null,false,3625,59.0,100.0


All information that does not describe a user, or a user-item interaction can therefore be removed

In [11]:
irelevant_columns = ["impression_time", "device_type", "article_ids_inview", "article_ids_clicked", "session_id", "next_read_time", "next_scroll_percentage"]
train_behaviors_df = train_behaviors_df.drop(irelevant_columns)
train_behaviors_df.head()

impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,i32,f32,f32,u32,bool,i8,i8,i8,bool
150528,null,25.0,null,143471,false,null,null,null,false
153070,9777492,26.0,100.0,151570,false,null,null,null,false
153071,9778623,125.0,100.0,151570,false,null,null,null,false
155586,9777492,134.0,100.0,161621,false,null,null,null,false
155588,9778769,24.0,100.0,161621,false,null,null,null,false


The remaining items are the ones that can be used. But already here we see that we have several features with lacking information. We should therefore treat this

### Account for Missing Values

We see here that alot of the behaviours contain missing values. The therefore have to either remove or replace the values

In [12]:
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(334250, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,235053,0,236553,0,0,311625,327911,325540,0


In [13]:
train_behaviors_df = train_behaviors_df.filter(train_behaviors_df["article_id"].is_not_null())
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99197, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,2526,0,0,92396,96964,95843,0


We see that of 70421, are there between 65-68000 missing values for gender, postcode and age. We therefore remove these as there is no use subsidizing them

In [14]:
train_behaviors_df = train_behaviors_df.drop(["gender", "postcode", "age"])
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99197, 7)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,is_subscriber
u32,u32,u32,u32,u32,u32,u32
0,0,0,2526,0,0,0


We still see that 2570/98967 rows are missing a scroll percentage. As this is very low (<3%) we can easily replace this. Intitially we just set scroll to 0

In [15]:
train_behaviors_df = train_behaviors_df.fill_null(strategy="zero")

### Account for Multiple Instances of the Same Article and User

By checking rows where the user_id and article_id are the same we see that we have 9855 instances where the user has read the same article multiple times

In [16]:
duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)

print(duplicates)

shape: (12_257, 3)
┌────────────┬─────────┬───────┐
│ article_id ┆ user_id ┆ count │
│ ---        ┆ ---     ┆ ---   │
│ i32        ┆ u32     ┆ u32   │
╞════════════╪═════════╪═══════╡
│ 9779520    ┆ 1614260 ┆ 2     │
│ 9788898    ┆ 644500  ┆ 2     │
│ 9773726    ┆ 266292  ┆ 2     │
│ 9785668    ┆ 2474986 ┆ 2     │
│ 9780561    ┆ 350401  ┆ 2     │
│ …          ┆ …       ┆ …     │
│ 9776303    ┆ 1009512 ┆ 2     │
│ 9721606    ┆ 2005504 ┆ 2     │
│ 9779538    ┆ 1312147 ┆ 2     │
│ 9771473    ┆ 1083980 ┆ 2     │
│ 9783965    ┆ 835650  ┆ 4     │
└────────────┴─────────┴───────┘


C:\Users\magnu\AppData\Local\Temp\ipykernel_17156\4233841252.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)


We see that we need to combine these duplicate rows. We therefore propose that for multiple instances of the same article and user, we combine the readtime and select the largest scroll percentage. This way we can preserve the data without having duplicates

In [17]:
dataProcesser = DataProcesser()
behaviors_df = dataProcesser.collaborative_filtering_preprocess()
train_df, test_df = dataProcesser.random_split(behaviors_df, test_ratio=0.2)
print(train_df.head())

shape: (5, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9779045    ┆ 2275401 ┆ 55.0           ┆ 100.0      │
│ 9780195    ┆ 2467656 ┆ 68.0           ┆ 100.0      │
│ 9785992    ┆ 2331745 ┆ 109.0          ┆ 100.0      │
│ 9777296    ┆ 2238237 ┆ 101.0          ┆ 100.0      │
│ 9786313    ┆ 554192  ┆ 2340.0         ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [18]:
recommender = UserBasedCollaborativeRecommender(train_df)
recommender.fit()

{2275401: [(14241, np.float64(0.9034312533266295)),
  (739976, np.float64(0.9034312533266295)),
  (24329, np.float64(0.9034312533266295)),
  (2213475, np.float64(0.9034312533266295)),
  (745747, np.float64(0.9034312533266295)),
  (791521, np.float64(0.9034312533266295)),
  (750378, np.float64(0.9034312533266295)),
  (1555282, np.float64(0.9034312533266295)),
  (1892109, np.float64(0.9034312533266295)),
  (1248366, np.float64(0.9034312533266295))],
 2467656: [(743885, np.float64(0.7840458244617614)),
  (1495359, np.float64(0.7840458244617614)),
  (677305, np.float64(0.7840458244617614)),
  (691539, np.float64(0.7840458244617614)),
  (914821, np.float64(0.7840458244617614)),
  (163374, np.float64(0.7840458244617614)),
  (2572936, np.float64(0.7840458244617613)),
  (2219788, np.float64(0.7840458244617613)),
  (2033355, np.float64(0.7840458244617613)),
  (1431118, np.float64(0.784045819621107))],
 2331745: [(2319135, np.float64(0.5252956439881362)),
  (1907419, np.float64(0.525295643988136

This first model just compares all artilces read by users when comparing users

In [59]:
binary_recommender = UserBasedCollaborativeRecommender(train_df, binary_model=True)
binary_recommender.fit()

{1889636: [(292099, np.float64(0.5773502691896258)),
  (1751491, np.float64(0.5773502691896258)),
  (1906874, np.float64(0.5773502691896258)),
  (2295463, np.float64(0.5773502691896258)),
  (66378, np.float64(0.5773502691896258)),
  (722501, np.float64(0.5773502691896258)),
  (364261, np.float64(0.4082482904638629)),
  (1192315, np.float64(0.4082482904638629)),
  (1503358, np.float64(0.4082482904638629)),
  (1606050, np.float64(0.4082482904638629))],
 343739: [(2186441, np.float64(0.4264014327112209)),
  (1936220, np.float64(0.40451991747794525)),
  (2302998, np.float64(0.3481553119113957)),
  (774439, np.float64(0.30151134457776363)),
  (1023907, np.float64(0.30151134457776363)),
  (1221334, np.float64(0.30151134457776363)),
  (1819384, np.float64(0.30151134457776363)),
  (1692553, np.float64(0.30151134457776363)),
  (1321793, np.float64(0.30151134457776363)),
  (2047439, np.float64(0.30151134457776363))],
 1076213: [(866952, np.float64(0.5773502691896258)),
  (807825, np.float64(0.57

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model Presentation

### Article Recommendation

In [19]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9778351, 9772453, 9771554, 9776497, 9776147]
reccomended for user  620796 :  [9783865, 9785049, 9785475, 9783379, 9765753]
reccomended for user  1067393 :  [9771113, 9774789, 9785986, 9785424, 9780849]
reccomended for user  1726258 :  []
reccomended for user  17205 :  [9780325]


In [20]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

NameError: name 'binary_recommender' is not defined

### Evaluation Scores

#### Without the Ability to Recommend Read Articles

The complex model only reccomending articles the user has not yet read

In [22]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.0019417475728155341),
 'NDCG@K': np.float64(0.022546168512007376)}

The binary reccomender model only reccomending articles the user has not yet read

In [23]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.001020408163265306),
 'NDCG@K': np.float64(0.00446524363159866)}

#### With the Ability to Recommend Previously Read Articles

The complex model reccomending articles the user, even if they have read them before

In [24]:
results = recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.0023529411764705885),
 'NDCG@K': np.float64(0.025861059941469613)}

The binary reccomender model reccomending articles the user, even if they have read them before

In [25]:
results = binary_recommender.evaluate_recommender(test_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.0012371134020618558),
 'NDCG@K': np.float64(0.014925879596339018)}

## Model Experimentation

In [26]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, 9782092, 9780815, None, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9782722, 9782131, 9783137, 9772508, 9773574, 9786649, 9754241, 9782361, 9778500, 9772029, 9786351, 9775489, 9779423, 9776508, 9789065, 9778302, 9770288, 9790052, 9778971, 9774899, 9783349, 9771576, 9773078, 9787264, 9777969, 9778369, 9789703, 9775697, 9785668, 9783655, 9777955, 9779248, 9779204, 9783628, 9783732, 9779182, 9780547, 9784273, 9777324, 9776864, 9780921, 9789248, 9781389, 9782836, 9781001, 9782884, 9775850, 9774516, 9788841, 9770102, 9790752, 9783891, 9787353, 9774430, 9782837, 9770327, 9779659, 9782517, 9777856, 9785732, 9781506, 9775824, 9784852, 9782635, 9772903, 9769996, 9777705, 9781838, 97

In [27]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9772045, 9774864, 9778448, 9774120, 9756075, 9771948, 9778351, 9774142, 9777856, 9771460, 9780815, 9776855, 9786718, 9780193, 9738729, 9775985, 9781875, 9776246, 9781887}
[9782722, 9782131, 9783137, 9772508, 9773574, 9786649, 9754241, 9782361, 9778500, 9772029, 9786351, 9775489, 9779423, 9776508, 9789065, 9778302, 9770288, 9790052, 9778971, 9774899, 9783349, 9771576, 9773078, 9787264, 9777969, 9778369, 9789703, 9775697, 9785668, 9783655, 9777955, 9779248, 9779204, 9783628, 9783732, 9779182, 9780547, 9784273, 9777324, 9776864, 9780921, 9789248, 9781389, 9782836, 9781001, 9782884, 9775850, 9774516, 9788841, 9790752, 9770102, 9783891, 9787353, 9774430, 9782837, 9770327, 9779659, 9777856, 9782517, 9785732, 9781506, 9775824, 9784852, 9772903, 9782635, 9769996, 9777705, 9781838, 9771948, 9773464, 9771125, 9769800, 9775596, 9779653, 9772099, 9785017, 9775800, 9676767, 9774972, 9771554, 9773846, 9782092, 9775763, 9771919, 9775713, 9711985, 9783803, 9774527, 9785260, 9786718, 9776071, 9772575,

### Model Evaluation
Performs and appends (to csv file `/evaluation_summary/model_overview.csv`) the model evaluation of precision@k, recall@k and fpr@k.

In [21]:
from utils.evaluation import perform_model_evaluation

matrics = perform_model_evaluation(recommender, test_df, k=5)
matrics

{'precision@k': np.float64(0.0035192280403008374),
 'recall@k': np.float64(0.006583872117151911),
 'fpr@k': np.float64(0.0019105603775412406)}

In [22]:
from utils.evaluation import append_model_metrics

append_model_metrics(matrics, "user based CF")

### Diversity Evaluation
Calculates the aggrigate diversity of the recommender model recommendations, and appends the result to the `/evaluation_summary/model_overview_diversity.csv`-file. 

In [ ]:
from utils.evaluation import aggregate_diversity
from utils.evaluation import append_aggregate_diversity

diversity = aggregate_diversity(recommender, articles_df, user_sample=1000)

print("Diversity")
print(diversity)

append_aggregate_diversity(diversity, "user_based_cf")

['article_id', 'title', 'subtitle', 'last_modified_time', 'premium', 'body', 'published_time', 'image_ids', 'article_type', 'url', 'ner_clusters', 'entity_groups', 'topics', 'category', 'subcategory', 'category_str', 'total_inviews', 'total_pageviews', 'total_read_time', 'sentiment_score', 'sentiment_label']
Diversity
0.05342848876458675


### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [ ]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "user_based", user_id=test_user_id, n=5)
footprint

[codecarbon INFO @ 12:14:16] [setup] RAM Tracking...
[codecarbon INFO @ 12:14:16] [setup] CPU Tracking...
[codecarbon WARNING @ 12:14:16] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon WARNING @ 12:14:17] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700H but we don't know it. Please contact us.
[codecarbon INFO @ 12:14:17] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 12:14:17] [setup] GPU Tracking...
[codecarbon INFO @ 12:14:17] No GPU found.
[codecarbon INFO @ 12:14:17] >>> Tracker's metadata:
[codecarbon INFO @ 12:14:17]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 12:14:17]   Python version: 3.11.9
[codecarbon INFO @ 12:14:17]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 12:14:17]   Available RAM : 15.731 GB
[codecarbon INFO @ 12:14:17]   CPU count: 20
[codecarbon INFO @ 12:14:17]   CPU model: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 12:14:17]   GPU count: None
[codecarbon INFO @ 12:14:17]   GPU model: None
[codecarbon INFO @ 12:14:21] Saving emissions data to file c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\demostrations\output\user_based_fit_

{'fit': ({1957222: [(934536, np.float64(0.9426891017067498)),
    (439129, np.float64(0.9147931928874817)),
    (2325549, np.float64(0.8690605169475955)),
    (262657, np.float64(0.6993805218030159)),
    (727268, np.float64(0.6615611763316869)),
    (1915689, np.float64(0.5347437343315455)),
    (66603, np.float64(0.5321020729321025)),
    (2523201, np.float64(0.49744498331481823)),
    (945074, np.float64(0.4815410732897871)),
    (881196, np.float64(0.448629653167123))],
   163652: [(1686572, np.float64(0.6068931561026079)),
    (1553127, np.float64(0.6068931561026079)),
    (1637884, np.float64(0.6068931561026079)),
    (1208893, np.float64(0.6068931561026079)),
    (594038, np.float64(0.6068931561026079)),
    (1470089, np.float64(0.6068931561026079)),
    (198059, np.float64(0.6068931561026079)),
    (482294, np.float64(0.6068931561026079)),
    (2347079, np.float64(0.6068931561026079)),
    (889788, np.float64(0.6068931561026079))],
   1734991: [(143807, np.float64(0.30450588970